# Speech Emotion Recognition Model

## 1. Import Libraries

In [1]:
import json
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 2. Load Dataset

In [2]:
def load_data(path):
    with open(path, 'r') as file:
        data = json.load(file)

    X = data['mfcc']
    y = np.array(data['label'])
    
    max_length = max(len(arr) for arr in X)
    padded_mfcc = pad_sequences(X, maxlen=max_length, padding='post', truncating='post')
    X = np.array(padded_mfcc)
    
    return X, y

In [3]:
# Path to json file
file_path = 'data.json'

# Load dataset
X, y = load_data(file_path)

In [4]:
print(X.shape)

(11318, 615, 13)


## 3. Prepare Dataset

In [5]:
def prepare_dataset(X, y, test_size, validation_size):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size)

    # Normalize data
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)

    std = np.where(std == 0, 1e-8, std)
    
    X_train = (X_train - mean)/std
    X_val = (X_val - mean)/std
    X_test = (X_test - mean)/std

    # Add new axis to X
    X_train= X_train[..., np.newaxis] 
    X_val = X_val[..., np.newaxis] 
    X_test = X_test[..., np.newaxis] 
    
    # One hot encoding
    y_train_encoded = tf.keras.utils.to_categorical(y_train)
    y_val_encoded = tf.keras.utils.to_categorical(y_val)
    y_test_encoded = tf.keras.utils.to_categorical(y_test)
    
    
    return X_train, X_val, X_test, y_train_encoded, y_val_encoded, y_test_encoded

In [6]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_dataset(X, y, 0.2, 0.2)


## 4. Build and Train Model

In [7]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
print(X_val.shape[1], X_val.shape[2], X_val.shape[3])
print(input_shape)
print(y_train.shape) 

615 13 1
(615, 13, 1)
(7243, 6)


In [37]:
model = tf.keras.Sequential()

# 1st CNN Layer
model.add(tf.keras.layers.Conv2D(32, (4,2), activation='relu', input_shape=input_shape))
model.add(tf.keras.layers.MaxPool2D((2,2), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

# 2nd CNN Layer
model.add(tf.keras.layers.Conv2D(64, (4,2), activation='relu'))
model.add(tf.keras.layers.MaxPool2D((2,2), padding='same'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

# 3rd CNN Layer
model.add(tf.keras.layers.Conv2D(128, (4,2), activation='relu'))
model.add(tf.keras.layers.MaxPool2D((2,2), padding='same', strides=(2,2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))


# RNN Layer
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))
model.add(tf.keras.layers.LSTM(64, return_sequences=True))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())

# Dense Layer
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.L1L2(0.01)))

# Output Layer
model.add(tf.keras.layers.Dense(6, activation='softmax'))

model.summary()


Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 612, 12, 32)       288       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 306, 6, 32)       0         
 g2D)                                                            
                                                                 
 batch_normalization_38 (Bat  (None, 306, 6, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_39 (Dropout)        (None, 306, 6, 32)        0         
                                                                 
 conv2d_33 (Conv2D)          (None, 303, 5, 64)        16448     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 152, 3, 64)     

In [38]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer,
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)

Epoch 1/100
227/227 [==============================] - 95s 399ms/step - loss: 9.2332 - accuracy: 0.3401 - val_loss: 3.9042 - val_accuracy: 0.1679
Epoch 2/100
227/227 [==============================] - 92s 404ms/step - loss: 3.1420 - accuracy: 0.4596 - val_loss: 3.3580 - val_accuracy: 0.2904
Epoch 3/100
227/227 [==============================] - 94s 415ms/step - loss: 2.8290 - accuracy: 0.5086 - val_loss: 2.8230 - val_accuracy: 0.5119
Epoch 4/100
227/227 [==============================] - 93s 411ms/step - loss: 2.6332 - accuracy: 0.5329 - val_loss: 2.4984 - val_accuracy: 0.4975
Epoch 5/100
227/227 [==============================] - 93s 411ms/step - loss: 2.4445 - accuracy: 0.5387 - val_loss: 2.4261 - val_accuracy: 0.5516
Epoch 6/100
227/227 [==============================] - 95s 419ms/step - loss: 2.3047 - accuracy: 0.5618 - val_loss: 2.2668 - val_accuracy: 0.5665
Epoch 7/100
227/227 [==============================] - 95s 418ms/step - loss: 2.1928 - accuracy: 0.5761 - val_loss: 2.2229 -

Epoch 57/100
227/227 [==============================] - 95s 420ms/step - loss: 1.2732 - accuracy: 0.8008 - val_loss: 1.7755 - val_accuracy: 0.6560
Epoch 58/100
227/227 [==============================] - 94s 414ms/step - loss: 1.2995 - accuracy: 0.7962 - val_loss: 1.7557 - val_accuracy: 0.6726
Epoch 59/100
227/227 [==============================] - 94s 415ms/step - loss: 1.2561 - accuracy: 0.8053 - val_loss: 1.7219 - val_accuracy: 0.6726
Epoch 60/100
227/227 [==============================] - 96s 421ms/step - loss: 1.2875 - accuracy: 0.8035 - val_loss: 1.7688 - val_accuracy: 0.6632
Epoch 61/100
227/227 [==============================] - 94s 415ms/step - loss: 1.2437 - accuracy: 0.8114 - val_loss: 1.8391 - val_accuracy: 0.6494
Epoch 62/100
227/227 [==============================] - 94s 416ms/step - loss: 1.2442 - accuracy: 0.8060 - val_loss: 1.7235 - val_accuracy: 0.6786
Epoch 63/100
227/227 [==============================] - 95s 418ms/step - loss: 1.2476 - accuracy: 0.8045 - val_loss: 1

## 5. Evaluating Model

In [40]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on test set is: {}".format(test_accuracy))

71/71 [==============================] - 5s 68ms/step - loss: 1.8110 - accuracy: 0.6731
Accuracy on test set is: 0.6731448769569397


## 6. Saving Model

In [41]:
# Save the model to a pickle file
model_path = 'model.pkl'

with open(model_path, 'wb') as f:
    pickle.dump(model, f)